# TastyTradeAPI – Ejemplo de uso en Jupyter Notebook

Este documento muestra cómo utilizar la **API de Tastytrade** a través de la librería `TastyTradeAPI` incluida en el proyecto.  

⚠️ **Nota importante**:  
En esta sesión demo no dispongo de una **cuenta de trading asociada**, por lo que **no es posible ejecutar órdenes reales** ni consultar ciertas métricas de cuenta.  
Aun así, todas las funciones están implementadas y acompañadas de **ejemplos de salida simulada**, por lo que se puede entender perfectamente cómo utilizarlas sin riesgo de error.  

La documentación principal de la API está incluida en el propio código (`api.py`), con explicaciones y ejemplos de cada bloque de funciones. 
 
---

# Índice

- [TastyTradeAPI – Ejemplo de uso en Jupyter Notebook](#tastytradeapi-ejemplo-de-uso-en-jupyter-notebook)
- [Estructura del proyecto](#estructura-del-proyecto)
- [Importar la API](#importar-la-api)
- [Login como cliente](#login-como-cliente)
- [Revisar si la sesión sigue activa](#revisar-si-la-sesion-sigue-activa)
- [Información de la cuenta](#informacion-de-la-cuenta)
  - [Información básica del cliente](#informacion-basica-del-cliente)
  - [Métricas de la cuenta](#metricas-de-la-cuenta)
  - [Datos de posiciones](#datos-de-posiciones)
  - [Datos de transacciones](#datos-de-transacciones)
- [Ejecutar órdenes](#ejecutar-ordenes)
- [Streamear datos de mercado históricos](#streamear-datos-de-mercado-historicos)
  - [Parámetros](#parametros)
  - [Ejemplo](#ejemplo)
- [Streamear datos en tiempo real](#streamear-datos-en-tiempo-real)
  - [Parámetros](#parametros-1)
  - [Respuesta](#respuesta)


---

## Estructura del proyecto

- **TastyTradeAPI/**  
  - `__init__.py` → inicializador del módulo.  
  - `api.py` → implementación de la API con todas las funciones documentadas. 

<br>

- **example_es.ipynb** → cuaderno de ejemplo (este documento) en español.
- **example_en.ipynb** → cuaderno de ejemplo en inglés.
- **requirements.txt** → librerías externas necesarias para ejecutar el proyecto.  
- **.env** → credenciales y configuración privada (usuario y contraseña de Tastytrade).  
- **venv/** → entorno virtual de Python.  




## Importar la API

Para trabajar con esta API debes tener el repositorio **TastyTradeAPI** en el mismo directorio de tu proyecto e importarlo como se muestra a continuación.

En el ejemplo se asigna el alias `ts`, aunque puedes utilizar cualquier nombre válido en Python.

Las credenciales (`usuario` y `contraseña`) corresponden al correo y la contraseña de tu cuenta de Tastytrade.  
Por seguridad, se recomienda cargarlas como **variables de entorno** en lugar de escribirlas directamente en el código. No obstante, también pueden pasarse como *strings* si lo prefieres.

In [1]:
from TastyTradeAPI.api import TastyTradeAPI as ts
import os
from dotenv import load_dotenv

load_dotenv()

USER = os.environ.get("MAIL")
PASS = os.environ.get("PASS")

## Login como cliente

Para inicializar el cliente de la API es necesario indicar el **usuario** y la **contraseña** de tu cuenta de Tastytrade.  

No es obligatorio tener una cuenta de trading abierta: basta con iniciar sesión.  
Ten en cuenta que, sin una cuenta activa, no podrás ejecutar operaciones ni consultar métricas principales, aunque sí es posible acceder a los **datos históricos de mercado**.

El cliente generado es un diccionario con la información de la sesión, por ejemplo (`el token no es real`):

In [7]:
api = ts()
client = api.Client(USER,PASS)

print(client)

{"session_token": "fz9a8b7c6d5e4f3g2h1i0jklmnopqrstuVWXY", "account_number": "No Account"}


## Revisar si la sesión sigue activa

La autenticación está basada en **tokens simples** (no en OAuth2), por lo que la sesión puede caducar tras unos minutos de inactividad.  

La función correspondiente devuelve `True` si el cliente sigue activo y `False` en caso contrario.  
En caso de que la sesión haya expirado, será necesario volver a **inicializar un nuevo cliente** con las credenciales.  


In [3]:
state = api.still_connected(client)

print(state)

True


## Información de la cuenta

Una vez iniciada la sesión, es posible consultar los datos básicos del cliente y las métricas principales de su cuenta.

### Información básica del cliente
Devuelve datos generales como **nombre**, **email**, **país de origen** y el **net worth** asociado al perfil.

In [4]:
info = api.client_info(client)

print(info)

{'name': 'Pablo Miñana', 'email': 'gandpablo04@gmail.com', 'citizenship_country': 'ESP', 'liquid_net_worth': 0}


### Métricas de la cuenta

Las métricas proporcionan una visión rápida del estado financiero de la cuenta asociada al cliente:

- **Balance** → saldo total disponible en la cuenta.  
- **Equity Buying Power** → poder de compra disponible para operar con acciones.  
- **Derivative Buying Power** → poder de compra disponible para operar con derivados (opciones, futuros).  
- **Liquidity** → liquidez actual de la cuenta, considerando activos y pasivos.  
- **Total Fees** → suma de todas las comisiones generadas hasta el momento.  

In [ ]:
balance = api.balance(client)

equityBP = api.equity_BP(client)

derivativeBP = api.derivative_BP(client)

liquidity = api.liquidity(client)

total_fees = api.total_fees(client)

print('Balance:', balance)
print('Equity BP:', equityBP)
print('Derivative BP:', derivativeBP)
print('Liquidity:', liquidity)
print('Total Fees:', total_fees)

```Ejemplo de salida```

```python
Balance: 12534.25
Equity BP: 10000.00
Derivative BP: 2500.00
Liquidity: 8750.50
Total Fees: 35.75
```

---

### Datos de posiciones

Estas funciones permiten consultar información detallada sobre las posiciones abiertas en la cuenta asociada al cliente de Tastytrade.

Cada posición incluye los siguientes campos:

- **symbol** → Ticker de la acción.  

- **quantity** → Cantidad de acciones en la posición.  

- **type** → Tipo de posición: `Long` o `Short`.  

- **profit** → Ganancia (>0) o pérdida (<0), calculada en función de la diferencia entre el precio actual (*mark*) y el precio de entrada (*cost basis*), ajustada por la cantidad y el multiplicador.  

  **Fórmulas:**  

  - **Posiciones *Long***  
  ```
   Profit = ( Mark − Cost Basis ) × Quantity × Multiplier
  ```

  - **Posiciones *Short***  
  ```
   Profit = ( Cost Basis − Mark ) × Quantity × Multiplier
  ```


- **avg_entry_price** → Precio promedio de entrada.  

- **current_price** → Precio actual de la acción.  

- **opened_at** → Fecha y hora de apertura de la posición.  

- **updated_at** → Última fecha y hora de actualización de la posición.  

La función devuelve un **diccionario**, donde la clave es el *ticker* y el valor es otro diccionario con toda la información de la posición correspondiente.


In [ ]:
positions = api.all_positions(client)

print(positions)

```Ejemplo de salida```

```python
{
  "AAPL": {
    "symbol": "AAPL",
    "quantity": 100,
    "type": "Long",
    "profit": 1250.00,
    "avg_entry_price": 150.00,
    "current_price": 162.50,
    "opened_at": "2025-09-10 14:30:00",
    "updated_at": "2025-09-17 09:15:00"
  },
  "TSLA": {
    "symbol": "TSLA",
    "quantity": 50,
    "type": "Short",
    "profit": -750.00,
    "avg_entry_price": 240.00,
    "current_price": 255.00,
    "opened_at": "2025-09-12 10:05:00",
    "updated_at": "2025-09-17 09:15:00"
  }
}
```
---

También es posible consultar una posición concreta a partir de su *ticker*.  

La función devuelve un **diccionario con la información de la posición** si existe, o `None` en caso contrario.  


In [ ]:
pos = api.check_position(client, "AAPL")

print(pos)

```Ejemplo de salida```

```python
{
  "symbol": "AAPL",
  "quantity": 100,
  "type": "Long",
  "profit": 1250.00,
  "avg_entry_price": 150.00,
  "current_price": 162.50,
  "opened_at": "2025-09-10 14:30:00",
  "updated_at": "2025-09-17 09:15:00"
}
```

---

### Datos de transacciones

Estas funciones permiten revisar las transacciones realizadas en la cuenta asociada al cliente de Tastytrade.  

Cada transacción incluye los siguientes campos:

- **id** → Identificador único de la transacción.  

- **transaction_type** → Tipo de transacción (ej. `Buy`, `Sell`, `Dividend`, `Fee`...).  

- **description** → Descripción de la operación (ej. *Buy to Open*).  

- **quantity** → Cantidad de acciones o contratos involucrados.  

- **price** → Precio unitario de ejecución.  

- **value** → Valor total de la transacción.  

- **date** → Fecha y hora de ejecución de la operación.  

La función devuelve un **diccionario** en el que la clave es el *ticker* y el valor es una **lista de transacciones** asociadas a ese símbolo.


In [ ]:
transactions = api.all_transactions(client)

print(transactions)

`Ejemplo de salida`

```python
{
  "AAPL": [
    {
      "id": "TX001",
      "transaction_type": "Buy",
      "description": "Buy to Open",
      "quantity": 100,
      "price": 150.00,
      "value": 15000.00,
      "date": "2025-09-10 14:30:00"
    }
  ],
  "TSLA": [
    {
      "id": "TX002",
      "transaction_type": "Sell",
      "description": "Sell to Close",
      "quantity": 50,
      "price": 240.00,
      "value": 12000.00,
      "date": "2025-09-12 10:05:00"
    }
  ]
}
```


También es posible consultar las transacciones asociadas a un *ticker* específico.  

La función devuelve una **lista de transacciones** si existen, o `None` en caso contrario

In [ ]:
txs = api.check_transaction(client, "AAPL")

print(txs)

`Ejemplo de salida`

```python
[
  {
    "id": "TX001",
    "transaction_type": "Buy",
    "description": "Buy to Open",
    "quantity": 100,
    "price": 150.00,
    "value": 15000.00,
    "date": "2025-09-10 14:30:00"
  }
]
```
---

## Ejecutar órdenes

Para dar una orden debes indicar:

- **symbol (ticker)**: El activo sobre el que operar, por ejemplo `"AAPL"`.  

- **quantity (val)**: El número de acciones o contratos.  

- **order-type**: Si la orden es `'Long'` (posición larga) o `'Short'` (posición corta).  

- **action**: Si quieres abrir una posición (`'Start'`) o cerrarla (`'End'`).  

- **time-in-force**: Determina cuánto tiempo permanece activa la orden:  
  - `'Day'` → válida solo hasta el cierre del mercado.  
  - `'GTC'` → permanece activa hasta que se ejecute o la canceles.  
  - `'GTD'` → válida hasta una fecha concreta (requiere parámetro adicional `gtc-date`).  
  - `'Ext'` → válida solo en horario extendido (equities).  
<br>

- **otype**: Define cómo se ejecuta la orden:  
  - `'Market'` → se ejecuta al mejor precio disponible, sin indicar precio.  
  - `'Limit'` → requiere indicar precio y se ejecuta si el mercado lo alcanza.  
  - `'Stop'` → se activa como market cuando se cumple un precio disparador.  
  - `'Stop Limit'` → se activa como limit cuando se cumple un precio disparador.  
<br>

Por ejemplo, para: Comprar 10 acciones de Apple (AAPL) con una orden de mercado (Market), válida hasta el cierre de hoy (Day).


In [ ]:
order_result = api.order(
    Client=client,
    ticker="AAPL",        
    val=10,               
    order_type="Long",    
    action="Start",       
    time_force="Day",     
    otype="Market"        
)

print(order_result)

`Ejemplo de salida`

```python
{
  "Status": "Filled",
  "Size": 10,
  "Fees": 1.25,
  "Commission": 0.50,
  "NewBuyingPower": 9850.00,
  "ReceivedAt": "2025-09-17 14:35:00"
}
```


Al ejecutar una orden, se devuelve un diccionario con los siguientes campos:

- **Status** → Estado de la orden (`Routed`, `Filled`, `Cancelled`, etc.).  

- **Size** → Cantidad enviada en la orden.  

- **Fees** → Tasas totales asociadas a la operación.  

- **Commission** → Comisión del broker.  

- **NewBuyingPower** → Poder de compra disponible tras la orden.  

- **ReceivedAt** → Fecha y hora en que la orden fue recibida. 

## Streamear datos de mercado históricos

Esta función se conecta con el proveedor de *market data* **DXFeed** (integrado en Tastytrade).  

⚠️ **Importante**: <br>
**Puede tardar varios segundos**. Dependiendo de la cantidad de tickers que solicites y la cantidad de datos, pero no es extraño que tarde `más de 20 segundos` para 50 datos de un solo ticker. Si pones muchos datos de muchas acciones puede incluso ejecutar por `más de 1 minuto`.

El flujo de trabajo es el siguiente:

1. Con el **cliente** se obtiene un *token* de DX.  

2. Con ese token se solicita vía **websocket** el histórico de velas (*candles*) de los **tickers** indicados.  

3. Los datos recibidos se formatean en un `DataFrame` con las variables elegidas.  

4. Finalmente, se devuelven en un diccionario donde cada clave es un ticker y el valor es su `DataFrame`.  

⚠️ **Limitación importante de DXFeed**:  
Por diseño, solo permite pedir datos desde el **momento actual hacia atrás** hasta el número máximo solicitado (`max_data`).  
No se pueden pedir intervalos arbitrarios (ej. de una fecha a otra).  
Por eso existe la función auxiliar `values_from_data`, que calcula cuántos registros necesitas para llegar hasta la fecha deseada.

---

##### Parámetros

- **client** → Cliente de tastytrade definido al principio.  

- **tickers** → Lista de símbolos de los activos que quieres consultar.  
  Ejemplo: `["AAPL", "MSFT"]`.  

- **interval** → Define la granularidad de los datos. Se pasa como número + unidad:  
  - `s` → segundos  
  - `m` → minutos  
  - `h` → horas  
  - `d` → días  
  - `w` → semanas  
  - `mo` → meses  
  Ejemplo: `"5m"` = velas de 5 minutos.  
<br>

- **vars** → Lista de variables que quieres recuperar. Valores disponibles (especificar las que quieres)
  - `'open'` → Apertura (`Open`) 
  - `'high'` → Máximo (`High`)  
  - `'low'` → Mínimo (`Low`)  
  - `'close'` → Cierre (`Close`)  
  - `'volume'` → Volumen (`Volume`)  
<br>

- **max_data** → Número máximo de registros a solicitar.  
  Normalmente se ajusta con la función `values_from_data` para obtener justo los datos necesarios hasta la fecha deseada.  


##### `Ejemplo`:
`Descargar 50 velas cada 5 minutos de AAPL y TSLA, solo open close y volume`

La función devuelve un **diccionario** con la siguiente estructura:

```python
{
    "AAPL": DataFrame con columnas [Open, Close, Volume, Date],
    "TSLA": DataFrame con columnas [Open, Close, Volume, Date],
}

In [5]:
tickers = ['AAPL','TSLA']
interval = '5m'
max_data= 50
vars = ['open', 'close','volume']

historical = api.get_historical(client, tickers, interval, vars,max_data)

In [8]:
historical['TSLA'].head(6)

,Open,Close,Volume,Date
25,421.39,421.4199,5301.0,2025-09-16 21:30:00
26,421.3875,421.3,4699.0,2025-09-16 21:25:00
27,421.201,421.345,5873.0,2025-09-16 21:20:00
28,421.6378,421.2003,5399.0,2025-09-16 21:15:00
29,421.359,421.6,5233.0,2025-09-16 21:10:00
30,421.39,421.31,6369.0,2025-09-16 21:05:00


Se puede calcular `max_data` según una fecha (`values_from_data`)

Esta función calcula automáticamente cuántos puntos (`max_data`) hay que pedir en `get_historical` para cubrir desde una **fecha inicial** hasta **ahora**, en función del intervalo.

Parámetros necesarios
- **interval** → tamaño de vela (`'5m'`, `'1h'`, `'1d'`, `'1w'`, `'1mo'`).  
- **date** → fecha inicial (`datetime`).

Devuelve un entero que luego se pasa como `max_data` en `get_historical` para obtener todos los datos desde esa fecha hasta el presente.

`Ejemplo:` Obtener velas diarias ('1d') desde el **21 de abril de 2025** hasta hoy:

In [9]:
from datetime import datetime, timezone

date = datetime(2025, 4, 21, 12, 0, tzinfo=timezone.utc)
needed = api.values_from_data("1d", date)

print('Como max_data usaremos: ',needed)

Como max_data usaremos:  150


## Streamear datos en tiempo real 

`RealTimeStreamer` es un contenedor sencillo para conectarse al **websocket de DXFeed** (vía Tastytrade) y recibir **cotizaciones en tiempo real** sin que el usuario tenga que gestionar la conexión manualmente. Está pensado para dashboards, bots o scripts que necesiten precios en tiempo real de varios símbolos.

**Devuelve** un diccionario con el **`askPrice`** de cada cotización indicada. 
**Conexión:** es **abierta y persistente**; por ello, en ocasiones pueden llegar **mensajes difusos o incompletos**, algunas conexiones pueden **tardar en abrirse o cerrarse**, y **no todos los símbolos** actualizan a la vez (algunos mensajes **pueden llegar con retraso** respecto a otros).

**Funcionamiento:** el objeto envía mensajes continuamente hasta que el usuario lo detenga; normalmente se debe usar con **bucles `while` (perpetuos)** o **`while` con `break` / `for`** si solo se quiere activo un tiempo establecido.


```Los datos están contenidos en la variable del streamer llamada data --> objeto.data```

---

#### Parámetros

- `client`: cliente de tasty trade que hemos definido al principio.  
- `tickers`: lista de símbolos, p. ej. `["AAPL", "TSLA"]`.  
- `verbose` *(bool, opcional; por defecto `False`)*: si está en `True`, **imprime todos los mensajes** del websocket y el estado de la conexión (útil para depurar).

---

#### Respuesta

- **`data`** *(dict)*: último **`askPrice`** por símbolo, p. ej.  
  ```python
  {"AAPL": 238.00, "MSFT": 510.05}
  ```

  >
  > **``` Esta es la variable con la que podrás trabajar```**  
  >




In [3]:
import time

tickers = ['AAPL','TSLA','MSFT','AMZN','GOOGL']   

stream = api.RealTimeStreamer(client,tickers)

stream.start()

for _ in range(10):
    data = stream.data
    print(data)
    time.sleep(1)
    
stream.stop()

{'AAPL': 238.0, 'TSLA': 423.42, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.25, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.39, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.48, 'MSFT': 510.39, 'AMZN': 233.95, 'GOOGL': 251.52}
{'AAPL': 238.0, 'TSLA': 423.5, 'MSFT': 510.36, 'AMZN': 233.95, 'GOOGL': 251.52}


In [6]:
tickers = ['AAPL']

stream = api.RealTimeStreamer(client,tickers,verbose=True)

stream.start()


while True:
    data = stream.data
    print(data)

    if data and data['AAPL'] > 238:
        print("Se ha cumplido la condición, deteniendo el stream.")
        stream.stop()
        break

    time.sleep(1)

-->Connection opened
Message received --> {"type":"SETUP","channel":0,"keepaliveTimeout":120,"acceptKeepaliveTimeout":120,"version":"1.0-1.2.3-20240912-171526"}
Message received --> {"type":"AUTH_STATE","channel":0,"state":"UNAUTHORIZED"}
Message received --> {"type":"AUTH_STATE","channel":0,"state":"AUTHORIZED","userId":"abcd1234-5678-90ab-cdef-112233445566"}
Message received --> {"type":"CHANNEL_OPENED","channel":3,"service":"FEED","parameters":{"contract":"AUTO","subFormat":"LIST"}}
Message received --> {"type":"FEED_CONFIG","channel":3,"dataFormat":"COMPACT","aggregationPeriod":0.1}
Message received --> {"type":"FEED_CONFIG","channel":3,"dataFormat":"COMPACT","aggregationPeriod":0.1,"eventFields":{"Quote":["askPrice","eventSymbol"]}}
Message received --> {"type":"FEED_DATA","channel":3,"data":["Quote",[241.15,"AAPL"]]}
{'AAPL': 241.15}
Se ha cumplido la condición, deteniendo el stream.
-->Connection closed
-->Streamer stopped


`La variable userId esta generada aleatoriamente para que no sea visible`